<a href="https://colab.research.google.com/github/sutanu27/Django_project/blob/master/NNScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import python libraries required in this example:
import numpy as np
import pandas as pd
import random
import math
from platform import python_version
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import inspect

In [3]:

df = pd.read_csv('/content/IRIS.csv')

In [4]:
data=df.values
One_hot_encoed_data=pd.get_dummies(df, columns = ['species'])
data=One_hot_encoed_data.values
X, y = data[:, :-3], data[:, 4:7]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [5]:
class NNetwork:

#Initialization of neurones and weights
  def __init__(self, no_of_inputs, *neurone_layers_array,act_function="sigmoid"):
    self.len_neurone_layers_array=len(neurone_layers_array)
    self.no_of_inputs= no_of_inputs
    self.total_layers=[]
    self.total_layers.append(no_of_inputs)
    self.total_layers+=neurone_layers_array
    self.no_of_outputs= neurone_layers_array[-1]
    self.neurone_layers_array=[]
    self.weight=[]
    self.bias=[]
    self.sum=[]
    self.act=[]
    self.pred=[]
    self.input=[]
    self.output=[]
    self.error=[None]*self.no_of_outputs
    self.act_function=act_function
    for layer in neurone_layers_array:
      self.neurone_layers_array.append(layer)
    self.set_weights(init_with='randint',ll=-3,ul=3)
    self.set_biases(init_with='randint',ll=-3,ul=3)

#Initialization of weights
  def set_weights(self,value=0,init_with='value',ll=0,ul=1):
      for k in range(len(self.total_layers)-1):
        self.weight.append([])
        self.weight[k]=np.random.uniform(ll,ul,(self.total_layers[k],self.total_layers[k+1]))  #to be modified

#Initialization of biases
  def set_biases(self,value=0,init_with='value',ll=0,ul=1):
    for i in range(self.len_neurone_layers_array):
      self.bias.append([])
      self.bias[i]=np.random.uniform(ll,ul,(1,self.neurone_layers_array[i]))  #to be modified

  def set_input(self,df=None):
    if(df==None):
      for i in range(self.no_of_inputs):
        self.__set_value(self.input,'randint',1,15)
    else:
      self.input=df

  def set_output(self,df=None):
    if(df==None):
      for i in range(self.no_of_outputs):
        self.__set_value(self.output,'randint',1,15)
    else:
      self.output=df

  def forward(self,inp,hidden_function_name='reLU',output_function_name='softmax'):
    self.sum=[]
    self.act=[]
    self.sum.append([])
    self.act.append([])
    self.act[0]=np.array(inp).T
    for k in range(len(self.total_layers)-1):
      self.sum.append([])
      self.sum[k+1]=np.matmul(self.act[k],self.weight[k])
      self.sum[k+1]=np.add(self.sum[k+1],self.bias[k])
      if(k<len(self.total_layers)-2):
        self.act.append([])
        self.act[k+1]=self.actFunction(self.sum[k+1],function_name=hidden_function_name)
      else:
        self.get_prediction(function_name=output_function_name)
      
    
  def get_prediction(self,function_name='sigmoid'):
    if(function_name=='softmax'):
      self.pred=self.actFunction(self.sum[-1],function_name='softmax')
    elif(function_name=='argmax'):
      self.pred=self.actFunction(self.sum[-1],function_name='argmax')
    elif(function_name=='sigmoid'):
      self.pred=self.actFunction(self.sum[-1],function_name='sigmoid')
    else:
      self.pred=self.sum[-1]

  def backward(self,output,lr=0.1,loss_function_name='cross_entropy', output_function_name='softmax',hidden_function_name='reLU'):
    dError_by_dAct=[]
    dAct_by_dSum=[]
    dSum_by_dAct=[]
    dSum_by_dWeight=[]
    dSum_by_dBias=[]
    dError_by_dWeight=[]
    dError_by_dBias=[]
    dError_by_dSum=[]
    dError_by_dAct.append([])
    dError_by_dSum.append([])
    dAct_by_dSum.append([])
    dSum_by_dWeight.append([])
    dSum_by_dBias.append([])
    dError_by_dWeight.append([])
    dError_by_dBias.append([])
    dError_by_dAct[0]=self.dErrorFunction(np.reshape(output,(1,self.total_layers[-1])),function_name=loss_function_name)
    dAct_by_dSum[0]=self.delActfunction(self.sum[-1],function_name=output_function_name)
    dSum_by_dBias[0]=np.ones((1,self.no_of_outputs))
    dSum_by_dWeight[0]=np.reshape(self.act[-1],(1,self.total_layers[-2]))
    if(output_function_name=='softmax'):
      dError_by_dSum[0] =np.matmul(dError_by_dAct[0],dAct_by_dSum[0])
    else:
      dError_by_dSum[0] =np.multiply(dError_by_dAct[0],dAct_by_dSum[0])
    dError_by_dBias[0]=np.multiply(dError_by_dSum[0],dSum_by_dBias[0])
    dError_by_dWeight[0]=np.dot(dSum_by_dWeight[0].T,dError_by_dSum[0])
    for k in range(self.len_neurone_layers_array-1,0,-1):
      dSum_by_dAct.insert(0,[])
      dError_by_dAct.insert(0,[])
      dError_by_dSum.insert(0,[])
      dAct_by_dSum.insert(0,[])
      dSum_by_dWeight.insert(0,[])
      dSum_by_dBias.insert(0,[])
      dError_by_dWeight.insert(0,[])
      dError_by_dBias.insert(0,[])
      dSum_by_dAct[0]=self.weight[k] #(dS(l+1)/dA(l))
      dAct_by_dSum[0]=np.reshape(self.delActfunction(self.sum[k],function_name=hidden_function_name),(1,self.total_layers[k]))#dA(l)/dS(l)
      dSum_by_dBias[0]=np.ones((1,self.total_layers[k]))
      dSum_by_dWeight[0]=np.reshape(self.act[k-1],(self.total_layers[k-1],1)) #dS(l)/dW(l)
      dError_by_dAct[0]=np.matmul(dError_by_dSum[1],dSum_by_dAct[0].T) #dE/dA(l)=(dE/dS(l+1))(dS(l+1)/dA(l))
      if(hidden_function_name=='softmax'):
        dError_by_dSum[0] =np.matmul(dError_by_dAct[0],dAct_by_dSum[0]) #dE/dS(l)=(dA(l)/dS(l))*(dE(l)/dA(l))
      else:
        dError_by_dSum[0] =np.multiply(dError_by_dAct[0],dAct_by_dSum[0])
      dError_by_dBias[0]=np.multiply(dError_by_dSum[0],dSum_by_dBias[0])
      dError_by_dWeight[0]=np.dot(dSum_by_dWeight[0],dError_by_dSum[0])
    self.Descent(dError_by_dWeight,dError_by_dBias,lr=lr)

  def actFunction(self,X,function_name='sigmoid',a=1,c=0):
    if(function_name=='sigmoid'):
      func = lambda x: (1/(1+np.exp(-x)))
    elif(function_name=='reLU'):
      func = lambda x: x*a+c if(x>0) else 0
    elif(function_name=='binary_step'):
      func = lambda x: a if(x>0) else 0
    elif(function_name=='linear'):
      func = lambda x: x*a
    elif(function_name=='tanh'):
      func = lambda x: (2/(1+np.exp(-2*x)))-1
    elif(function_name=='swish'):
      func = lambda x: x/(1+np.exp(-x))
    elif(function_name=='argmax'):
      ind=np.argmax(X)
      out=[0]*self.no_of_outputs
      out[ind]=1
      return out
    elif(function_name=='softmax'):
      vec=np.vectorize(np.exp)
      e_x=vec(X)
      return e_x/np.sum(e_x)
    else:
      func= lambda x:x
    vec=np.vectorize(func)
    return vec(X)

  def delActfunction(self,X,function_name='sigmoid',a=1,c=0):
    if(function_name=='sigmoid'):
      func = lambda x: np.exp(-x)/((1+np.exp(-x))**2)
    elif(function_name=='reLU'):
      func = lambda x: a if(x>0) else 0
    elif(function_name=='binary_step'):
      return  [0]*len(X)
    elif(function_name=='linear'):
      return [a]*len(X)
    elif(function_name=='tanh'):
      func = lambda x: (2*np.exp(x))/(np.exp(2*x)+1)
    elif(function_name=='swish'):
      func = lambda x: (np.exp(-x)*(x+1)+1)/(1+np.exp(-x))**2
    elif(function_name=='softmax'):
      vec=np.vectorize(np.exp)
      e_x=vec(X)
      e_x=e_x/np.sum(e_x)
      diag=np.diag(e_x[0])
      ee=np.matmul(e_x.T,e_x)
      return np.subtract(diag,ee)
    else:
      return 1
    vec=np.vectorize(func)
    return vec(X)

  def errorFunction(self,output,function_name='MSE'):
    self.error=np.subtract(output,self.pred)
    if(function_name=='MSE'):
      return np.matmul(self.error,self.error)/2
    elif(function_name=='RMSE'):
      return np.sqrt(np.matmul(self.error,self.error))
    elif(function_name=='cross_entropy'):
      func= lambda x,y:-1*x*np.log(y)
      vec=np.vectorize(func)
      return np.sum(vec(output,self.pred))
    elif(function_name=='cross_entropy_binary'):
      func= lambda x,y:-1*x*np.log(y)-(1-x)*np.log(1-y)
      vec=np.vectorize(func)
      return np.sum(vec(output,self.pred))
    else:
      return np.sum(-1*e if e<0 else e for e in self.error)

  def dErrorFunction(self,output,function_name='MSE',a=1,c=0):
    if(function_name=='MSE'):
      return [ -1*e for e in self.error]
    elif(function_name=='RMSE'):
      RMSE=math.sqrt(sum( e*e for e in self.error)/2)
      return  [ -1*e/(2*RMSE) for e in self.error]
    elif(function_name=='cross_entropy'):
      func= lambda x,y:-1*x/y
      vec=np.vectorize(func)
      return vec(output,self.pred)
    elif(function_name=='cross_entropy_binary'):
      func= lambda x,y:(1-x)/(1-y)-x/y
      vec=np.vectorize(func)
      return vec(output,self.pred)
    else:
      return [ -1 if e>0 else 1 for e in self.error ]

  def __set_value(self,dict,init_with,ll=0,ul=0,value=0):
    if (init_with=='rand'):
      dict.append(random.uniform(ll,ul))
    elif(init_with=='randint'):
      dict.append(random.randint(ll,ul))
    elif(init_with=='one'):
      dict.append(1)
    else:
      dict.append(value)

  def Descent(self,delWeight,delBias,lr=0.1):
    self.weight=[ np.subtract(self.weight[k],lr*delWeight[k]) for k in range(len(self.weight))]
    self.bias=[ np.subtract(self.bias[k],lr*delBias[k]) for k in range(len(self.bias))]

  def train(self,x_train,y_train,loss_function_name='cross_entropy',hidden_function_name='sigmoid',output_function_name='softmax',lr=0.1):
    batch_error=0
    for i in range(len(x_train)):
      self.forward(x_train[i],hidden_function_name=hidden_function_name,output_function_name=output_function_name)
      batch_error+=self.errorFunction(y_train[i],function_name=loss_function_name)
      self.backward(y_train[i],loss_function_name=loss_function_name,hidden_function_name=hidden_function_name,output_function_name=output_function_name,lr=lr)
    return batch_error
  
  def cost(self,x_train,y_train,loss_function_name='cross_entropy',hidden_function_name='sigmoid',output_function_name='softmax',lr=0.1):
    self.forward(x_train,hidden_function_name=hidden_function_name,output_function_name=output_function_name)
    J=self.errorFunction(y_train,function_name=loss_function_name)
    return J

  def test(self,x_test):
    input_size=x_test.shape[0]
    y_pred=[]
    for i in range(input_size):
      self.forward(x_test[i],output_function_name='argmax')
      y_pred.append(self.pred)
    return np.array(y_pred)

  def printN(self):
    print(str(self.no_of_inputs) +" "+ str(self.neurone_layers_array))
    print('weights:')
    for w in self.weight:
      print(str(w))
    print('biases:')
    for b in self.bias:
      print(str(b))
    print('inputs:'+str(self.input))
    print('Sums:'+str(self.sum))
    print('Act:'+str(self.act))
    print('pred:'+str(self.pred))
    

In [6]:
def create_mini_batches(X, y, batch_size):
  mini_batches = []
  data = np.hstack((X, y))
  np.random.shuffle(data)
  n_minibatches = data.shape[0] // batch_size
  i = 0
  for i in range(n_minibatches ):
    mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
    X_mini = mini_batch[:, :-y.shape[-1]]
    Y_mini = mini_batch[:, -y.shape[-1]:]
    mini_batches.append((X_mini, Y_mini))
  if((data.shape[0] % batch_size) != 0):
    mini_batch = data[(i+1) * batch_size:data.shape[0]]
    X_mini = mini_batch[:, :-y.shape[-1]]
    Y_mini = mini_batch[:, y.shape[-1]:-1]
    mini_batches.append((X_mini, Y_mini))
  return mini_batches


In [7]:
batch_size=20
mini_batches = create_mini_batches(X_train, y_train, batch_size)

In [ ]:
y_test

In [ ]:
#Gradient descent
epoch=500
input_size=X_train.shape[-1]
output_size=y_train.shape[-1]
NN=NNetwork(input_size,6,6,output_size)

for i in range(epoch):
  error=NN.train(X_train, y_train,lr=0.2)
  print('Epoch:'+str(i)+', Error :'+str(error))

y_pred=NN.test(X_test)
print(accuracy_score(y_test, y_pred))

In [9]:
#Mini Batch Gradient descent
epoch=200
batch_size=20
input_size=X_train.shape[-1]
output_size=y_train.shape[-1]
NN=NNetwork(input_size,10,20,10,output_size)

for i in range(epoch):
  mini_batches = create_mini_batches(X_train, y_train, batch_size)
  k=1
  for mini_batch in mini_batches:
    X_mini, y_mini = mini_batch
    error=NN.train(X_mini, y_mini,lr=0.2)
    print('Epoch:'+str(i)+',batch:'+str(k)+', Error :'+str(error))
    k+=1


y_pred=NN.test(X_test)
print(accuracy_score(y_test, y_pred))


Epoch:0,batch:1, Error :34.943196472262
Epoch:0,batch:2, Error :22.285493093022904
Epoch:0,batch:3, Error :21.561545605909505
Epoch:0,batch:4, Error :15.322376457619427
Epoch:0,batch:5, Error :14.016539405831892
Epoch:0,batch:6, Error :14.33225864124743
Epoch:1,batch:1, Error :11.073923948464357
Epoch:1,batch:2, Error :10.258506196266099
Epoch:1,batch:3, Error :15.645615402747715
Epoch:1,batch:4, Error :11.975117418791346
Epoch:1,batch:5, Error :14.612493089475395
Epoch:1,batch:6, Error :12.602185907707241
Epoch:2,batch:1, Error :19.379497307802463
Epoch:2,batch:2, Error :14.348536337649142
Epoch:2,batch:3, Error :11.376879762598916
Epoch:2,batch:4, Error :11.439945471138978
Epoch:2,batch:5, Error :12.889503828128822
Epoch:2,batch:6, Error :10.109216427571258
Epoch:3,batch:1, Error :11.198694570028845
Epoch:3,batch:2, Error :11.626063194640732
Epoch:3,batch:3, Error :8.084373388271993
Epoch:3,batch:4, Error :14.368041010620681
Epoch:3,batch:5, Error :8.906121136956825
Epoch:3,batch:6, 

In [ ]:
a=NNetwork(3,6,5,4)

In [ ]:
a.printN()

3 [6, 5, 4]
weights:
[[-1.66975435  2.41277962 -1.59697482 -2.52609135  0.75826345 -2.81432267]
 [-2.79612095  1.43323437 -2.28524403  2.23214006 -1.00661812 -0.80752715]
 [-2.6573698  -0.72345283 -1.69723392 -1.67102098  2.58031271 -0.52323425]]
[[-1.57389912  2.78117904 -1.95327513  1.25883616 -2.17434036]
 [ 2.68217674  2.97434231  1.90340449  2.58224919  0.37910239]
 [-0.2879427  -2.66318768  0.97326753 -1.19922703 -2.1617849 ]
 [-2.05886523 -1.5333683  -1.12371896  0.31947968 -0.76398869]
 [ 1.96164965 -2.93008688 -0.64818772  0.13750258  2.11215573]
 [-1.63947266  0.2708192  -0.93180757 -2.00116838 -0.16674661]]
[[-2.94851522 -1.79061869  2.32764736 -1.93921306]
 [ 2.76638396 -0.76322491 -2.88098479 -0.14078034]
 [ 0.878599   -0.62159184  0.73043818 -1.78174523]
 [ 1.88898746  0.37158884 -0.73601066  1.6268665 ]
 [-0.16608959  0.15743883 -0.70149333  1.01439691]]
biases:
[[ 0.31637606  0.57448015 -0.05279988 -2.79756217 -0.02654081  1.46658211]]
[[-0.29541075  0.5180133   1.23340

In [ ]:
-1.10753691*7+2.77902709*1+1.69909824*3

0.12356343999999986

In [ ]:
a.error

array([[ 9.94576818e-01, -7.03578413e-02, -4.25729604e-05,
        -9.24176403e-01]])

In [ ]:
a.train([[7,1,3],[4,3,5]],[[1,0,0,0],[0,0,1,0]])

In [ ]:
a.backward()